In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

# from fastai import *
# from fastai.text import *
import kenlm
from tqdm import tqdm
import fastText
import pandas as pd
from bleu import *

In [2]:
# #loading openai GPT

# import torch, os
# from pytorch_pretrained_bert import OpenAIGPTTokenizer, OpenAIGPTLMHeadModel
# import numpy as np

# lm_model_special_tokens = ["<POS>","<NEG>","<END>"]
# lm_tokenizer = OpenAIGPTTokenizer.from_pretrained('openai-gpt', special_tokens=lm_model_special_tokens)
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# lm_model = OpenAIGPTLMHeadModel.from_pretrained('openai-gpt', num_special_tokens=len(lm_model_special_tokens))

# path = os.path.join(os.getcwd(), "./log_yelp_lm/yelp_language_model_2.bin")
# lm_model_state_dict = torch.load(path)
# lm_model.load_state_dict(lm_model_state_dict)
# lm_model.to(device)
# lm_model.eval()

# lm_loss = torch.nn.CrossEntropyLoss(ignore_index=-1, reduction='none')

# def calculate_ppl_gpt(sentence_batch, sentiment):
#     # tokenize the sentences
#     tokenized_ids = [None for i in range(len(sentence_batch))]
    
#     for i in range(len(sentence_batch)):
#         tkns = lm_tokenizer.tokenize(sentiment + ' ' +sentence_batch[i])
#         tokenized_ids[i] = lm_tokenizer.convert_tokens_to_ids(tkns)
#     sen_lengths = [len(x) for x in tokenized_ids]
#     max_sen_lenght = max(sen_lengths)
    
#     n_batch = len(sentence_batch)
#     input_ids = np.zeros( shape=(n_batch, max_sen_lenght), dtype=np.int64)
#     lm_labels = np.full(shape=(n_batch, max_sen_lenght), fill_value=-1)
    
#     for i, tokens in enumerate(tokenized_ids):
#         input_ids[i, :len(tokens)] = tokens
#         lm_labels[i, :len(tokens)] = tokens[1:] + [lm_tokenizer.special_tokens["<END>"]]
    
#     input_ids = torch.tensor(input_ids).to(device)
#     lm_labels = torch.tensor(lm_labels).to(device)
#     with torch.no_grad():
#         lm_pred = lm_model(input_ids)
#     loss_val = lm_loss(lm_pred.view(-1, lm_pred.size(-1)), lm_labels.view(-1))
#     normalized_loss = loss_val.view(n_batch,-1).sum(dim= -1) / torch.tensor(sen_lengths, dtype=torch.float32).to(device)
#     #normalized_loss = loss_val.view(n_batch,-1).sum(dim= -1)
#     ppl = torch.exp(normalized_loss)
#     return  ppl.tolist() 
	
# def calculate_dataset_ppl(input_sentences,correct_sentiment,bs=16):
#     ppl = []
#     steps = len(input_sentences) // bs
#     for i in range(steps + 1):
#         if i != steps:
#             inp = input_sentences[i * bs : i * bs + bs]
#             ppl.extend(calculate_ppl(inp, sentiment=sentiment))
#         else:
#             inp = input_sentences[i * bs: len(input_sentences)]
#             ppl.extend(calculate_ppl(inp, sentiment=sentiment))
#     return sum(ppl) / len(ppl)


Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [4]:
#bert classifier
import torch, os
from tqdm import trange

from pytorch_pretrained_bert.file_utils import PYTORCH_PRETRAINED_BERT_CACHE
from pytorch_pretrained_bert.modeling import BertForSequenceClassification, BertConfig, WEIGHTS_NAME, CONFIG_NAME
from pytorch_pretrained_bert.tokenization import BertTokenizer

model_cls = BertForSequenceClassification.from_pretrained("./bert_classifier/yelp", num_labels=2)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

model_cls.to('cuda')
model_cls.eval()

max_seq_len=70
sm = torch.nn.Softmax(dim=-1)

def evaluate_dev_set(input_sentences, labels, bs=32):
    """
    To evaluate whole dataset and return accuracy
    """
    ids = []
    segment_ids = []
    input_masks = []
    pred_lt = []
    for sen in input_sentences:
        text_tokens = tokenizer.tokenize(sen)
        tokens = ["[CLS]"] + text_tokens + ["[SEP]"]
        temp_ids = tokenizer.convert_tokens_to_ids(tokens)
        input_mask = [1] * len(temp_ids)
        segment_id = [0] * len(temp_ids)
        padding = [0] * (max_seq_len - len(temp_ids))

        temp_ids += padding
        input_mask += padding
        segment_id += padding
        
        ids.append(temp_ids)
        input_masks.append(input_mask)
        segment_ids.append(segment_id)
    
    ids = torch.tensor(ids).to('cuda')
    segment_ids = torch.tensor(segment_ids).to('cuda')
    input_masks = torch.tensor(input_masks).to('cuda')
    
    steps = len(ids) // bs
    
    for i in trange(steps+1):
        if i == steps:
            temp_ids = ids[i * bs : len(ids)]
            temp_segment_ids = segment_ids[i * bs: len(ids)]
            temp_input_masks = input_masks[i * bs: len(ids)]
        else:
            temp_ids = ids[i * bs : i * bs + bs]
            temp_segment_ids = segment_ids[i * bs: i * bs + bs]
            temp_input_masks = input_masks[i * bs: i * bs + bs]
        
        with torch.no_grad():
            preds = sm(model_cls(temp_ids, temp_segment_ids, temp_input_masks))
        
        #preds = preds.view(-1,bs)
        try:
            args = torch.argmax(preds, dim=-1)
            pred_lt.extend(args.tolist())
        except RuntimeError:
            pass
    accuracy = sum(np.array(pred_lt) == np.array(labels)) / len(labels)
    
    return accuracy, pred_lt

In [8]:
# import torch, os
# from pytorch_pretrained_bert import GPT2Tokenizer, GPT2Model, GPT2LMHeadModel

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# import logging
# logging.basicConfig(level=logging.INFO)

# gpt2_lm_tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
# gpt2_lm_model = GPT2LMHeadModel.from_pretrained('gpt2')
# path = os.path.join(os.getcwd(), "./GPT2/yelp_language_model_1.bin")
# gpt2_lm_model_state_dict = torch.load(path)
# gpt2_lm_model.load_state_dict(gpt2_lm_model_state_dict)
# gpt2_lm_model.to(device)
# gpt2_lm_model.eval()

# lm_loss = torch.nn.CrossEntropyLoss(ignore_index=-1, reduction='none')
# def calculate_ppl_gpt2(sentence_batch):
#     # tokenize the sentences
#     tokenized_ids = [None for i in range(len(sentence_batch))]
    
#     for i in range(len(sentence_batch)):
#         tokenized_ids[i] = gpt2_lm_tokenizer.encode(sentence_batch[i])
        
#     sen_lengths = [len(x) for x in tokenized_ids]
#     max_sen_lenght = max(sen_lengths)
    
#     n_batch = len(sentence_batch)
#     input_ids = np.zeros( shape=(n_batch, max_sen_lenght), dtype=np.int64)
#     lm_labels = np.full(shape=(n_batch, max_sen_lenght), fill_value=-1)
    
#     for i, tokens in enumerate(tokenized_ids):
#         input_ids[i, :len(tokens)] = tokens
#         lm_labels[i, :len(tokens)-1] = tokens[1:] 
    
#     input_ids = torch.tensor(input_ids).to(device)
#     lm_labels = torch.tensor(lm_labels).to(device)
#     with torch.no_grad():
#         lm_pred = gpt2_lm_model(input_ids)
#     loss_val = lm_loss(lm_pred[0].view(-1, lm_pred[0].size(-1)), lm_labels.view(-1))
#     normalized_loss = loss_val.view(n_batch,-1).sum(dim= -1) / torch.tensor(sen_lengths, dtype=torch.float32).to(device)
#     #normalized_loss = loss_val.view(n_batch,-1).sum(dim= -1)
#     ppl = torch.exp(normalized_loss)
#     return  ppl.tolist() 

from pytorch_pretrained_bert import GPT2Tokenizer, GPT2Model, GPT2LMHeadModel

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
import logging
logging.basicConfig(level=logging.INFO)

lm_tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
lm_model = GPT2LMHeadModel.from_pretrained('gpt2')
path = os.path.join(os.getcwd(), "GPT2/yelp_language_model_1.bin")
lm_model_state_dict = torch.load(path)
lm_model.load_state_dict(lm_model_state_dict)
lm_model.to(device)
lm_model.eval()

lm_loss = torch.nn.CrossEntropyLoss(ignore_index=-1, reduction='none')


def calculate_ppl_gpt2(sentence_batch, bs=16):
    # tokenize the sentences
    tokenized_ids = [None for i in range(len(sentence_batch))]
    ppl = [None for i in range(len(sentence_batch))]
    
    for i in range(len(sentence_batch)):
        tokenized_ids[i] = lm_tokenizer.encode(sentence_batch[i])
        
    sen_lengths = [len(x) for x in tokenized_ids]
    max_sen_length = max(sen_lengths)
    
    n_batch = len(sentence_batch)
    input_ids = np.zeros( shape=(n_batch, max_sen_length), dtype=np.int64)
    lm_labels = np.full(shape=(n_batch, max_sen_length), fill_value=-1)
    
    for i, tokens in enumerate(tokenized_ids):
        input_ids[i, :len(tokens)] = tokens
        lm_labels[i, :len(tokens)-1] = tokens[1:] 
    
    input_ids = torch.tensor(input_ids)#.to(device)
    lm_labels = torch.tensor(lm_labels)#.to(device)
    
    steps = n_batch // bs
    
    for i in range(steps+1):
        
        if i == steps:
            temp_input_ids = input_ids[i * bs : n_batch]
            temp_lm_labels = lm_labels[i * bs : n_batch]
            temp_sen_lengths = sen_lengths[i * bs : n_batch]
        else:
            temp_input_ids = input_ids[i * bs : i * bs + bs]
            temp_lm_labels = lm_labels[i * bs : i * bs + bs]
            temp_sen_lengths = sen_lengths[i * bs : i * bs + bs]
            
        temp_input_ids = temp_input_ids.to('cuda')
        temp_lm_labels = temp_lm_labels.to('cuda')
            
        with torch.no_grad():
            lm_pred = lm_model(temp_input_ids)
            
        loss_val = lm_loss(lm_pred[0].view(-1, lm_pred[0].size(-1)), temp_lm_labels.view(-1))
        normalized_loss = loss_val.view(len(temp_input_ids),-1).sum(dim= -1) / torch.tensor(temp_sen_lengths, dtype=torch.float32).to(device)
        tmp_ppl = torch.exp(normalized_loss)
        ppl[i * bs: i * bs + len(temp_input_ids)] = tmp_ppl.tolist()
    
    return  ppl


INFO:pytorch_pretrained_bert.tokenization_gpt2:loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-vocab.json from cache at /home/ubuntu/.pytorch_pretrained_bert/f2808208f9bec2320371a9f5f891c184ae0b674ef866b79c58177067d15732dd.1512018be4ba4e8726e41b9145129dc30651ea4fec86aa61f4b9f40bf94eac71
INFO:pytorch_pretrained_bert.tokenization_gpt2:loading merges file https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-merges.txt from cache at /home/ubuntu/.pytorch_pretrained_bert/d629f792e430b3c76a1291bb2766b0a047e36fae0588f9dbc1ae51decdff691b.70bec105b4158ed9a1747fea67a43f5dee97855c64d62b6ec3742f4cfdb5feda
INFO:pytorch_pretrained_bert.modeling_gpt2:loading weights file https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-pytorch_model.bin from cache at /home/ubuntu/.pytorch_pretrained_bert/4295d67f022061768f4adc386234dbdb781c814c39662dd1662221c309962c55.778cf36f5c4e5d94c8cd9cefcf2a580c8643570eb327f0d4a1f007fab2acbdf1
INFO:pytorch_pretrained_bert.modeling_gpt

In [9]:
calculate_ppl_gpt2(["totally random lol", "this is a good sentance"])

[116.72223663330078, 81.89402770996094]

In [10]:
# LOADING MODELS

# #fastai classifier 
# path = Path('fastaimodels')

# data_clas = load_data(path, 'data_clas.pkl', bs=48)
# fastai_classifier = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5)
# fastai_classifier.load_encoder('yelp_fine_tuned_enc')
# fastai_classifier.load('yelp_classifier')

# #fastai lm
# data_lm = load_data(path, 'data_lm.pkl', bs=48)
# fastai_lm = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.3)
# fastai_lm.load_encoder('yelp_fine_tuned_enc')
# fastai_lm.load('yelp_fine_tuned')

#fasttext classifier
classifier_model = fastText.load_model('fasttextmodel/model_yelp.bin')

#kenlm lm
kenlm_lm = kenlm.Model('kenlmmodel/yelp.arpa')


In [9]:
# # fastai lm check
# print("\n".join(fastai_lm.predict("hey man", 20, temperature=0.75) for _ in range(2)))

# # fastai ppl functions
# def perplexitylm(sentance):
#     tokens = sentance.split()
#     str_list = []
#     for i, x in enumerate(tokens):
#         str1 = " ".join(tokens[:i+1])
#         str_list.append(str1)
#     prob = []

#     for i in range(0, len(str_list)-1):
#         xb, yb = data_lm.one_item(str_list[i])
#         #print(xb, yb)
#         p1=fastai_lm.pred_batch(batch=(xb,yb))[0][-1]
#         prob.append(p1[data_lm.one_item(tokens[i+1])[0][0][-1].item()].item())
    
#     perplexity = torch.exp(-torch.mean(torch.log(torch.tensor(prob)))).item()
    
#     return perplexity

In [13]:
df = pd.read_csv('yelp_all_model_prediction_ref1.csv', header = None)
label = 0
label_str = '__label__0'

list_sentences = df[1:len(df)].values.tolist()

list_sentences_source = []
list_sentences_human = []
for list_sentance in list_sentences:
    list_sentences_source.append(list_sentance[0])
    list_sentences_human.append(list_sentance[-1])

matrics1 = []
for i in tqdm(range(0, len(list_sentences[0]))):
    bleu_s = 0
    bleu_r = 0
    fasttext_c = 0
    kenlm_ppl = 0
    gpt_ppl = 0
    gpt2_ppl = 0
    #fastai_c = 0
    #fastai_ppl = 0
    
    sentences = []
    for j in range(0, len(list_sentences)):
        sentences.append(list_sentences[j][i])
        
    fasttext_labels = classifier_model.predict(sentences)
    
    total_sentences = len(sentences)

    bleu_s = get_bleu(list_sentences_source, sentences)
    bleu_r = get_bleu(list_sentences_human, sentences)

    for _, sentence in enumerate(sentences):
#         bleu_s += sentence_bleu([list_sentences_source[_]], sentence)
#         bleu_r += sentence_bleu([list_sentences_human[_]], sentence)
        
        if(fasttext_labels[0][_][0]==label_str):
            fasttext_c += 1
        kenlm_ppl += kenlm_lm.perplexity(sentence)
        
    labels_list = [label] * len(sentences)
    bert_accuracy, pred_label_list = evaluate_dev_set(sentences, labels_list)
    
    ppl_list_gpt2 = calculate_ppl_gpt2(sentences)
    
#     for j in range(0, len(ppl_list_gpt1)):
#         gpt_ppl += ppl_list_gpt1[j]
    for j in range(0, len(ppl_list_gpt2)):
        gpt2_ppl += ppl_list_gpt2[j]
        
#         fastai_label = fastai_classifier.predict(sentence)[1].item()
#         if(fastai_label == label):
#             fastai_c += 1
#         fastai_ppl += perplexitylm(sentence)
    matrics1.append([bleu_s , bleu_r , fasttext_c/total_sentences , kenlm_ppl/total_sentences, bert_accuracy, gpt2_ppl/len(ppl_list_gpt2)])
    
    

100%|██████████| 25/25 [02:58<00:00,  7.15s/it]


In [14]:
matrics1

[[100.0,
  58.76047151776541,
  0.052,
  63.416433129944586,
  0.008,
  22.210357120990754],
 [49.842279939134116,
  35.59684165362755,
  0.76,
  76.13768818849276,
  0.708,
  72.25274747371674],
 [81.05700836941423,
  48.870981193551025,
  0.078,
  89.72583258482746,
  0.078,
  110.92477409696579],
 [59.56135976022242,
  40.978338773986565,
  0.472,
  163.29706247774084,
  0.42,
  216.50014221334456],
 [23.64798342833085,
  18.964238104842654,
  0.942,
  7.864442458850198,
  0.966,
  38.179245446681975],
 [15.161083237506915,
  13.302010626509182,
  0.158,
  246.93935677608124,
  0.14,
  117.38152573871612],
 [55.70282939740121,
  38.44826753017345,
  0.87,
  101.97370205084117,
  0.894,
  71.80047267532349],
 [59.495961266331896,
  41.03778516267212,
  0.88,
  176.07347929461463,
  0.89,
  136.54219307804107],
 [70.09340434198477,
  52.9945021277317,
  0.9,
  81.74441315516009,
  0.956,
  31.26398136138916],
 [87.48103866259444,
  58.77862094962845,
  0.542,
  76.77049880755,
  0.498

In [15]:
df = pd.read_csv('yelp_all_model_prediction_ref0.csv', header = None)
label = 1
label_str = '__label__1'

list_sentences = df[1:len(df)].values.tolist()

list_sentences_source = []
list_sentences_human = []
for list_sentance in list_sentences:
    list_sentences_source.append(list_sentance[0])
    list_sentences_human.append(list_sentance[-1])

matrics0 = []
for i in tqdm(range(0, len(list_sentences[0]))):
    bleu_s = 0
    bleu_r = 0
    fasttext_c = 0
    kenlm_ppl = 0
    #gpt_ppl = 0
    gpt2_ppl = 0
    #fastai_c = 0
    #fastai_ppl = 0
    
    sentences = []
    for j in range(0, len(list_sentences)):
        sentences.append(list_sentences[j][i])
        
    fasttext_labels = classifier_model.predict(sentences)
    
    total_sentences = len(sentences)
    
    bleu_s = get_bleu(list_sentences_source, sentences)
    bleu_r = get_bleu(list_sentences_human, sentences)
    
    for _, sentence in enumerate(sentences):
#         bleu_s += sentence_bleu([list_sentences_source[_]], sentence)
#         bleu_r += sentence_bleu([list_sentences_human[_]], sentence)
        if(fasttext_labels[0][_][0]==label_str):
            fasttext_c += 1
        kenlm_ppl += kenlm_lm.perplexity(sentence)
        
    labels_list = [label] * len(sentences)
    bert_accuracy, pred_label_list = evaluate_dev_set(sentences, labels_list)
    
#     ppl_list_gpt1 = calculate_ppl_gpt(sentences, sentiment="<POS>")
    ppl_list_gpt2 = calculate_ppl_gpt2(sentences)
    
#     for j in range(0, len(ppl_list_gpt1)):
#         gpt_ppl += ppl_list_gpt1[j]
    for j in range(0, len(ppl_list_gpt2)):
        gpt2_ppl += ppl_list_gpt2[j]
        
#         fastai_label = fastai_classifier.predict(sentence)[1].item()
#         if(fastai_label == label):
#             fastai_c += 1
#         fastai_ppl += perplexitylm(sentence)
    matrics0.append([bleu_s , bleu_r , fasttext_c/total_sentences , kenlm_ppl/total_sentences, bert_accuracy, gpt2_ppl/len(ppl_list_gpt2)])
    
    

100%|██████████| 25/25 [02:57<00:00,  7.11s/it]


In [16]:
[print(i) for i in matrics0]

[100.0, 57.333733790819196, 0.038, 82.70006055162344, 0.018, 25.718697894096376]
[46.06305753940541, 35.33115113972936, 0.776, 65.26673933836403, 0.754, 73.3109280166626]
[74.89798211161994, 45.43406373618434, 0.112, 121.80164262862935, 0.086, 120.84426799345016]
[54.95903979143561, 37.19520640834666, 0.54, 155.16837282553405, 0.522, 194.67919037866594]
[28.074403766790894, 21.831363013918125, 0.9, 8.633996946904736, 0.96, 39.437523376464846]
[68.64257406970076, 44.89914441052, 0.842, 246.93935677608124, 0.86, 117.38152573871612]
[57.659395398845184, 38.326106429498466, 0.848, 98.36773751385932, 0.816, 79.8402370018959]
[56.46444244003326, 37.83799796003757, 0.876, 61.34533349825818, 0.898, 43.45983244132996]
[71.96655090402137, 50.98927581200577, 0.802, 113.42534715416198, 0.822, 45.99616223359108]
[89.96628493958603, 55.7498597309706, 0.328, 124.7409502603151, 0.3, 35.14967751455307]
[70.16107205739775, 49.576709189366305, 0.73, 171.04383920565138, 0.782, 86.66571102786064]
[81.35080

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [17]:
[print(i) for i in matrics1]

[100.0, 58.76047151776541, 0.052, 63.416433129944586, 0.008, 22.210357120990754]
[49.842279939134116, 35.59684165362755, 0.76, 76.13768818849276, 0.708, 72.25274747371674]
[81.05700836941423, 48.870981193551025, 0.078, 89.72583258482746, 0.078, 110.92477409696579]
[59.56135976022242, 40.978338773986565, 0.472, 163.29706247774084, 0.42, 216.50014221334456]
[23.64798342833085, 18.964238104842654, 0.942, 7.864442458850198, 0.966, 38.179245446681975]
[15.161083237506915, 13.302010626509182, 0.158, 246.93935677608124, 0.14, 117.38152573871612]
[55.70282939740121, 38.44826753017345, 0.87, 101.97370205084117, 0.894, 71.80047267532349]
[59.495961266331896, 41.03778516267212, 0.88, 176.07347929461463, 0.89, 136.54219307804107]
[70.09340434198477, 52.9945021277317, 0.9, 81.74441315516009, 0.956, 31.26398136138916]
[87.48103866259444, 58.77862094962845, 0.542, 76.77049880755, 0.498, 27.26058078622818]
[70.71006476905083, 49.42216530890736, 0.432, 127.82366577900062, 0.462, 55.94305122995377]
[77.

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [22]:
matricsavg = (np.array(matrics0)+np.array(matrics1))/2

In [23]:
matricsavg = matricsavg.tolist()

In [24]:
df_res0 = pd.DataFrame(matrics0, columns=['BLEU_source','BLEU_human','fasttext_classifier','klm_ppl', 'BERT_classifier', 'gpt2_ppl'])

In [25]:
df_res1 = pd.DataFrame(matrics1, columns=['BLEU_source','BLEU_human','fasttext_classifier','klm_ppl', 'BERT_classifier', 'gpt2_ppl'])

In [26]:
df_resavg = pd.DataFrame(matricsavg, columns=['BLEU_source','BLEU_human','fasttext_classifier','klm_ppl', 'BERT_classifier', 'gpt2_ppl'])

In [17]:
#gleu_list = [0.076314, 0.044249, 0.059500, 0.047856, 0.008023, 0.098865, 0.063792, 0.071165, 0.116518, 0.098625, 1.000000]

In [ ]:
# openaigpt_ppl0 = 
# openaigpt_ppl1 = 

# bertclassifier0 = 
# bertclassifier1 = 

# gleu0 =
# gleu1 =

# #df_res.insert(loc=0, column='GLEU_score', value=gleu_list)
# df_res0.insert(column='openaigpt_ppl', value=models_list[0])
# df_res1.insert(column='openaigpt_ppl', value=models_list[0])
# df_resavg.insert(column='openaigpt_ppl', value=models_list[0])

In [27]:
models_list = df[0:1].values.tolist()
#df_res.insert(loc=0, column='GLEU_score', value=gleu_list)
df_res0.insert(loc=0, column='model', value=models_list[0])
df_res1.insert(loc=0, column='model', value=models_list[0])
df_resavg.insert(loc=0, column='model', value=models_list[0])

In [28]:
df_resavg

,model,BLEU_source,BLEU_human,fasttext_classifier,klm_ppl,BERT_classifier,gpt2_ppl
0,Source,100.000000,58.047103,0.045,73.058247,0.013,23.964528
1,CROSSALIGNED,47.952669,35.463996,0.768,70.702214,0.731,72.781838
2,STYLEEMBEDDING,77.977495,47.152522,0.095,105.763738,0.082,115.884521
3,MULTIDECODER,57.260200,39.086773,0.506,159.232718,0.471,205.589666
4,RETRIEVEONLY,25.861194,20.397801,0.921,8.249220,0.963,38.808384
5,TEMPLATEBASED,41.901829,29.100578,0.500,246.939357,0.500,117.381526
6,DELETEONLY,56.681112,38.387187,0.859,100.170720,0.855,75.820355
7,DELETEANDRETRIEVE,57.980202,39.437892,0.878,118.709406,0.894,90.001013
8,BERT_DEL,71.029978,51.991889,0.851,97.584880,0.889,38.630072
9,SEL_DEL,88.723662,57.264240,0.435,100.755725,0.399,31.205129


In [ ]:
# [print(i) for i in df_resavg['BLEU_source'].tolist()]

In [36]:
df_res0.to_csv('matrics/yelp/matrics_yelp_all_model_prediction_0.csv')

In [37]:
df_res1.to_csv('matrics/yelp/matrics_yelp_all_model_prediction_1.csv')

In [38]:
df_resavg.to_csv('matrics/yelp/matrics_yelp_all_model_prediction_avg.csv')